# Installation Guide

Install CCDPApy package into google colab

In [1]:
"""
Installation code for private respository
"""
# Mount drive
'''from google.colab import drive
drive.mount('/content/drive')

import os

# Path to CCDPApy
# _ccdpapy_dir = '/content/drive/MyDrive/Yudai Fukae/Three CLs/CCDPApy-master'
_ccdpapy_dir = '/content/drive/MyDrive/Work/Undergrad Research/Yudai Fukae/Three CLs/CCDPApy-master' #for YAL

os.chdir(_ccdpapy_dir)
print(os.getcwd())'''

"from google.colab import drive\ndrive.mount('/content/drive')\n\nimport os\n\n# Path to CCDPApy\n# _ccdpapy_dir = '/content/drive/MyDrive/Yudai Fukae/Three CLs/CCDPApy-master'\n_ccdpapy_dir = '/content/drive/MyDrive/Work/Undergrad Research/Yudai Fukae/Three CLs/CCDPApy-master' #for YAL\n\nos.chdir(_ccdpapy_dir)\nprint(os.getcwd())"

In [2]:
"""
Installation code for public respository
"""
# # Mount drive
# from google.colab import drive
# drive.mount('/content/drive')

# import os
# # Create folders for CCDPApy Tutorial
# _ccdpapy_tutorial_dir = '/content/drive/MyDrive/CCDPApy_Tutorial'
# if os.path.exists(_ccdpapy_tutorial_dir) == False:
#   os.mkdir(_ccdpapy_tutorial_dir)
# os.chdir(_ccdpapy_tutorial_dir)

# # clone git respository
# ! git clone https://github.com/lu000285/CCDPApy.git

# # cd into package folder
# _ccdpapy_dir = '/content/drive/MyDrive/CCDPApy_Tutorial/CCDPApy'
# os.chdir(_ccdpapy_dir)
# print(os.getcwd())

'\nInstallation code for public respository\n'

# Overview Of Data Processing

## Diagrams

## Template of Input File

# CellLine class

To store and aggregate data for mutiple manufacturing runs in different Cell Line.

## ___CellLine___ Class
Store BioProcess objects for data aggregation.

In [3]:
from CCDPApy import CellLine

cell_line = CellLine()

## ___CellLine.add_bio_process___ method

Parameters
----------
    bio_process : python object
    BioProcess object

In [4]:
cell_line.add_bio_process(bio_process=bio_process)

NameError: name 'bio_process' is not defined

## ___CellLine.disp_cell_lines()___ method

Display Cell Line Name and Experiment ID stored in CellLine Class.

In [ ]:
cell_line.disp_cell_lines()

Cell Line: Sample CL1
Experiment 1: Sample CL1_1


## ___CellLine.get_cell_line(cl_name)___ method

Return dictionary of BioProcess object stored in CellLine class.

Returns
-------
    python dictionary
        {'cell line name': BioProcess object}


In [ ]:
cl1 = cell_line.get_cell_line(cl_name='Sample CL1')

# Case Studies; Three Industrial Cell Lines

## Tutorial For Cell Line 1

Cell Line 1 has __3 differnet experiments__.<br>
Cell Line 1 has __feed concentrations__ but no measurements of __concentrations after feeding__.
__Glutamine__ feed is added in Cell Line 1.

In [ ]:
from CCDPApy import CellLine    # used to aggregate all cell lines and experiments data
from CCDPApy import bioprocess_pipeline # used for data process of one experiment

data_file_1 = ['CL1_1.xlsx', 'CL1_2.xlsx', 'CL1_3.xlsx']
key_cl1 = {'use_feed_conc': True,
           'use_conc_after_feed': False,
           'spc_list': ['Glucose','Lactate','Glutamine','Asparagine','Aspartate'],
           'polyreg': True, 'polyorder_file': 'polynomial_order_1.xlsx',
           'rollreg': True, 'rollreg_order': 3, 'rollreg_window': 6,
           }

cell_line = CellLine()

for input in data_file_1:
    cl_1 = bioprocess_pipeline(input_file_name=input, **key_cl1)
    cell_line.add_bio_process(bio_process=cl_1)

CL1_1.xlsx imported.
CL1_2.xlsx imported.
CL1_3.xlsx imported.


## Tutorial For Cell Line 2

Cell Line 2 has __3 experiments__.<br>
Cell Line 2 has measurements of __concentrations after feeding__ but does't have __feed concentraions__.<br>
__Glucose__ feed is added in Cell Line 2.

In [ ]:
from CCDPApy import CellLine    # used to aggregate all cell lines and experiments data
from CCDPApy import bioprocess_pipeline # used for data process of one experiment

data_file_2 = ['CL2_1.xlsx', 'CL2_2.xlsx', 'CL2_3.xlsx']
key_cl2 = {'use_feed_conc': False,
           'use_conc_after_feed': True,
           'spc_list': ['Glucose','Lactate','Glutamine','Asparagine','Aspartate'],
           'polyreg': True, 'polyorder_file': 'polynomial_order_2.xlsx',
           'rollreg': True, 'rollreg_order': 3, 'rollreg_window': 6,
           }
for input in data_file_2:
    cl_2 = bioprocess_pipeline(input_file_name=input, **key_cl2)
    cell_line.add_bio_process(bio_process=cl_2)


CL2_1.xlsx imported.
CL2_2.xlsx imported.
CL2_3.xlsx imported.


## Tutorial For Cell Line 3

Cell Line 3 has __3 experiments__.<br>
Cell Line 3 doesn't have both measurements of __concentrations after feeding__ and __feed concentraions__.<br>
For __glucose__, Cell Line 3 has the __calculated cumulative profile__ in Measured Data.<br>
__Glutamine__ feed is added in Cell Line 3.

In [ ]:
data_file_3 = ['CL3_1.xlsx', 'CL3_2.xlsx', 'CL3_3.xlsx']
key_cl3 = {'use_feed_conc': False,
           'use_conc_after_feed': False,
           'spc_list': ['Glucose','Lactate','Glutamine','Asparagine','Aspartate'],
           'all_method': True
           }

for input in data_file_3:
    cl_3 = bioprocess_pipeline(input_file_name=input, **key_cl3)
    cell_line.add_bio_process(bio_process=cl_3)

CL3_1.xlsx imported.
CL3_2.xlsx imported.
CL3_3.xlsx imported.


# Interactive Plot

In [ ]:
cell_line.interactive_plot()

# Save Processed Data as an Excel

### ___CellLine.save_excel(cell_line, file_name)____ method

Save each bioprocess data in the same cell line as an Excel file.<br>
Please include '.xlsx'.<br>
Do not include rolling regression data. To save the rolling regression data,
use save_excel_rollreg method. 

Parameters
----------
cell_line : str
    Cell Line name.
file_name : str
    File name.
    Please include '.xlsx'.

In [ ]:
cell_line.get_cell_line_list()

['Sample CL1', 'Sample CL2', 'Sample CL3']

In [ ]:
cell_line.save_excel(cell_line='Sample CL1', file_name='cl_1')
cell_line.save_excel(cell_line='Sample CL2', file_name='cl_2')
cell_line.save_excel(cell_line='Sample CL3', file_name='cl_3')

cl_1.xlsx saved
cl_2.xlsx saved
cl_3.xlsx saved


## Save Rolling Regression Data as an Excel

### ___CellLine.save_excel_rollreg(cell_line, file_name)___ method

Save each rolling regression data in the same cell line as an Excel file.<br>
Please include '.xlsx'.<br>
Do not include other bioprocess data. To save other data,
use save_excel method.

Parameters
----------
cell_line : str
    Cell Line name.
file_name : str
    File name.
    Please include '.xlsx'.

In [ ]:
cell_line.save_excel_rollreg(cell_line='Sample CL1', file_name='cl_1_rollreg')
cell_line.save_excel_rollreg(cell_line='Sample CL2', file_name='cl_2_rollreg')
cell_line.save_excel_rollreg(cell_line='Sample CL3', file_name='cl_3_rollreg')

cl_1_rollreg.xlsx saved
cl_2_rollreg.xlsx saved
cl_3_rollreg.xlsx saved
